In [ ]:
# SECTION 1: IMPLEMENTATION

import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
AdjMatrix = np.zeros(shape=(8297,8297),dtype=int)
count = 0
with open('Wiki-Vote.txt','r') as f:
    for line in f:
        if line.startswith('#'):
            continue
        i,j = line.split('\t')
        AdjMatrix[int(i)-1,int(j)-1] = 1
        count += 1
# Q14
        
AdjMatrix_full = np.ones(shape=(8297,8297),dtype=float)
bytesperelement = AdjMatrix_full.itemsize
total_memory_MB = (8297 * 8297 * bytesperelement) / (1024*1024)
print(f"Memory Cost(with operations we need even more space):\t{total_memory_MB} MB")
# print(AdjMatrix_full.nbytes/(1024*1024))

# Q15
sparsity_degree = 1 - (count / (8297 * 8297))
print(f"sparsity:\t{sparsity_degree * 100} %")

# Q16
AdjMatrixT = AdjMatrix.T
print(f"symmetric?\t{np.array_equal(AdjMatrix,AdjMatrixT)}")


Memory Cost(with operations we need even more space):	525.209114074707 MB
sparsity:	99.84937727309922 %
symmetric?	False


For having a dense graph we need 525MB memory which may increase by doing operations.
Our graph matrix is too sparse.(99.84% zeros)
Since the edges are not two-sided, as we expect the symmetric check gives False as result.

In [ ]:
# SECTION 2: PAGERANK
# Q17
def PageRank(graph,p = 0.85,nodes = 8297):
    J = np.ones(shape=(nodes,nodes),dtype=float)
    S = np.zeros(shape=(nodes,nodes),dtype=float)
    for row in range(nodes):
        cj = 0
        for element in range(nodes):
            cj += graph[row,element]
        for element in range(nodes):
            if(cj != 0):
                S[row,element] = graph[row,element] / cj
            else:
                S[row,element] = 1 / nodes
    
    G = np.add((p * S) , (((1-p) * J) / nodes))
    GT = G.T
    x0 = np.ones(shape=nodes,dtype=float) / nodes
    while(True):
        x_new = GT @ x0
        diff = abs(np.max(x_new) - np.max(x0))
        if (diff < 0.000001):
            break
        x0 = x_new
    return x_new

# A = np.array([[0,1,0,0],[1,0,1,0],[1,0,0,0],[1,1,1,0]])
# vec = PageRank(A.T,0.85,4)


# Q18
vec = PageRank(AdjMatrix)
sorted_indices = np.argsort(vec)[::-1]

for i in range(5):
   print(f"User_ID: {sorted_indices[i]+1}\t(Score:\t{vec[sorted_indices[i]]})") 

    
    
    

    

User_ID: 4037	(Score:	0.004348467975133605)
User_ID: 15	(Score:	0.003472848431843667)
User_ID: 6634	(Score:	0.0033268565775298646)
User_ID: 2625	(Score:	0.0030997772239488375)
User_ID: 2398	(Score:	0.0024603701359481303)


In [ ]:
# SECTION 3: HITS Algorithm
# Q19
def HITS(A, max_iter=100,nodes = 8297 ,tol=1e-6):

    h = np.ones(nodes)
    a = np.ones(nodes)

    for _ in range(max_iter):
        a_new = A.T @ h
        a_new /= np.linalg.norm(a_new)

        h_new = A @ a_new
        h_new /= np.linalg.norm(h_new)

        if np.linalg.norm(a_new - a) < tol:
            break

        a, h = a_new, h_new

    return a, h

# Q20
Authority,Hub = HITS(AdjMatrix)

sorted_indices_a = np.argsort(Authority)[::-1]
sorted_indices_h = np.argsort(Hub)[::-1]

print("Authorities TOP Five:")
for i in range(5):
   print(f"User_ID: {sorted_indices_a[i]+1}")
print('\n----------------------------------\n')
print("Hubs TOP Five:")
for i in range(5):
   print(f"User_ID: {sorted_indices_h[i]+1}")

Authorities TOP Five:
User_ID: 2398
User_ID: 4037
User_ID: 3352
User_ID: 1549
User_ID: 762

----------------------------------

Hubs TOP Five:
User_ID: 2565
User_ID: 766
User_ID: 2688
User_ID: 457
User_ID: 1166


In [ ]:
# SECTION 4: Compare
# Q21
def votes_calculator(graph,nodes = 8297):
    votes_list = np.zeros(shape=nodes, dtype=float)
    for i in range(nodes):
        sum_row = 0
        for j in range(nodes):
            sum_row += graph[i,j]
        votes_list[i] = sum_row
    return votes_list

vt_lst = votes_calculator(AdjMatrixT)
sorted_indices_votes = np.argsort(vt_lst)[::-1]

for i in range(5):
   print(f"User_ID: {sorted_indices_votes[i]+1}\t(VOTES:\t{int(vt_lst[sorted_indices_votes[i]])})") 


User_ID: 4037	(VOTES:	457)
User_ID: 15	(VOTES:	361)
User_ID: 2398	(VOTES:	340)
User_ID: 2625	(VOTES:	331)
User_ID: 1297	(VOTES:	309)


Difference Between Algorithms and raw votes calculating: The difference is where we care about independences of columns(Rows) space and calculate the significant eigen and singular values(and vectors) to decide about matrix more efficient. But in raw calculations we see all the columns(Rows) and their values as same as the others(which is not wise!).

So as we expect, the result of raw comparing will be different from HITS and PageRank Results, generally.

Raw calculation is equivalent to compute AT . 1. it means we ignore eigen values of ATA which affects our matrix. So in raw calculation we will lost important properties of matrix(Graph).

In [32]:
# Section 4_2: Q22
A = csr_matrix(AdjMatrixT)
U,S,Vt = svds(A,k=10)

idx = S.argsort()[::-1]
S = S[idx]
U = U[:, idx]
Vt = Vt[idx, :]

singular_val = S[0]
eig_val = singular_val * singular_val
print(f"greatest singular value:\t{singular_val}\ngreatest eigenvalue:\t{eig_val}")

u1 = U[:, 0]
v1 = Vt[0, :]

sorted_indices_v = np.argsort(v1)[::-1]
sorted_indices_u = np.argsort(u1)[::-1]

print("U1/Authority TOP Five:")
for i in range(5):
   print(f"User_ID: {sorted_indices_u[i]+1}")
print('\n----------------------------------\n')
print("V1/Hubs TOP Five:")
for i in range(5):
   print(f"User_ID: {sorted_indices_v[i]+1}")
   
# cosine check Also

def cosine_similarity(x, y):
    return (x @ y) / (np.linalg.norm(x) * np.linalg.norm(y))
 
cos_au1 = cosine_similarity(Authority,u1)
cos_hv1= cosine_similarity(Hub,v1)
u1_tol = np.arccos(cos_au1)
v1_tol = np.arccos(cos_hv1)
print('---------------------------------------------')
print(f"tetha Between u1 and Authority ranking:\t{u1_tol}")
print(f"tetha Between v1 and Hub ranking:\t{v1_tol}")


greatest singular value:	103.18761071380561
greatest eigenvalue:	10647.68300482389
U1/Authority TOP Five:
User_ID: 2398
User_ID: 4037
User_ID: 3352
User_ID: 1549
User_ID: 762

----------------------------------

V1/Hubs TOP Five:
User_ID: 2565
User_ID: 766
User_ID: 2688
User_ID: 457
User_ID: 1166
---------------------------------------------
tetha Between u1 and Authority ranking:	1.5027287287092082e-06
tetha Between v1 and Hub ranking:	9.746316329102802e-07


As we can see, The Result for first row of vt is similar to Hubs ranking and The Result For first column of U is similar to Authority ranking!
**note: we rearranged U, sigma and vt because the algorithm is ascending but we need descending to see greatest singular-value First.

Since the tetha values are near to zero, we can say they are extremely similar!